In [ ]:
Cliente= 'Comex'
cuenta = '141796229342716'
access_token='EAAFLWQphDb4BACPjUReLZB3TtW6Lyz41v3BArzZBNbj2rmzWODiiIAiRar53ljgFdP5Qe0N7aq6xwQZCnyZAt3saj61FrtDOmnP2C6SKc25zm9QlXqhhovzqu2sZCyGaSxrzdqTVcW5s9pTQnmbPmEWO37G3gf0MZByNbASerLqwZDZD'

In [ ]:
ayer = datetime.date(2020,1,1)
ahora = datetime.date(2021,1,16)
print(ayer)
print(ahora)

#ahora = datetime.datetime.utcnow() + datetime.timedelta(days=2)
#print(ahora)

#ayer = ahora - datetime.timedelta(days=15)
#print(ayer.date())


unixtime1 = time.mktime(ayer.timetuple())
print(unixtime1)

unixtime2 = time.mktime(ahora.timetuple())
print(unixtime2)

In [ ]:
        base='https://graph.facebook.com/v8.0/'
        #access_token='EAAFLWQphDb4BAOfw4GP5Ol3ZCvL407twiKC5Edd4ugs8mhoM7kAv7Ma0HI10fSTCLrEZBEm1KHJZAvVfc6I8usXNqmhYKlcKMeBT7pEYXvtaGYgy7SYEYvNQO0svOIbZA9BgpOLK4p4iXfPRONiuyl2uF5SxxZCz9AoZBXKZACtuwZDZD'

        #url=base+cuenta+'conversations?fields=messages.limit(100){message,created_time,from}&limit=500&access_token=' + str(access_token)
        #url=base+cuenta+'?fields=posts{likes.summary(true),created_time,id,from,is_popular,is_published,picture,shares,story,story_tags,subscribed,message,comments{id,created_time,from,like_count,message,comments{id,from,created_time,like_count,message}}}&access_token=' + str(access_token)
        url=base+cuenta+'?fields=posts.since('+str(unixtime1)+').until('+str(unixtime2)+'){status_type,created_time,id,from,is_popular,is_published,picture,shares,story,story_tags,subscribed,permalink_url,message,comments.limit(0).summary(true),likes.limit(0).summary(true),reactions.limit(0).summary(true)}&access_token=' + str(access_token)
        print(url)
        data = requests.get(url).json()['posts']
        #print(data)
        post = []
        comentarios = []

        a = 1
        while(True):
            try:
                for datos in data['data']:
                    post.append(datos)
                    print(datos)
                    # Attempt to make a request to the next page of data, if it exists.
                #print(data['conversations']['paging']['next'])
                #time.sleep(1)

                url = data['paging']['next'].encode('utf-8')
                print('url')
                print(url)
                data = requests.get(url).json()
                #print(data)

                a+=1
            except KeyError:
                # When there are no more pages (['paging']['next']), break from the
                # loop and end the script.
                break
            #print(a)

        df_facebok_post = pd.DataFrame(post)
        #df_facebok_post
        history_facebook_post = pd.read_sql("select * from [IPE.DW].[dbo].[Informe.Facebook_post] where Cliente = '%s' and convert(date,[Fecha]) BETWEEN '%s' AND '%s'" %(Cliente,ayer,ahora),conn)
        #                                    and Fecha Between " % Cliente,conn)
        #history_facebook_post.shape


        df_facebok_post['id'] = df_facebok_post.id.apply(lambda x: x if not pd.isnull(x) else 0)
        df_facebok_post['from'] = df_facebok_post['from'].apply(lambda x: x if not pd.isnull(x) else '')
        df_facebok_post['message'] = df_facebok_post.message.apply(lambda x: x if not pd.isnull(x) else '')
        #df_facebok_post['story'] = df_facebok_post.story.apply(lambda x: x if not pd.isnull(x) else '')
        #df_facebok_post['story_tags'] = df_facebok_post.story_tags.apply(lambda x: x if not pd.isnull(x) else '')
        df_facebok_post['permalink_url'] = df_facebok_post.permalink_url.apply(lambda x: x if not pd.isnull(x) else '')
        df_facebok_post['status_type'] = df_facebok_post.status_type.apply(lambda x: x if not pd.isnull(x) else '')
        df_facebok_post['shares'] = df_facebok_post.shares.apply(lambda x: x if not pd.isnull(x) else {'count':'0'})
        df_facebok_post['likes'] = df_facebok_post.likes.apply(lambda x: x if not pd.isnull(x) else {"summary": {"total_count": 0}})

        def cod_postid(id):
            subguion = '_' 
            index_guion = id.index(subguion)    
            longitud= len(id) 
            cod_postid=int(id[index_guion+1:longitud])
            return cod_postid

        def quitar_guion(fecha):
            fecha = str(fecha)
            fechanew = fecha.replace('-', '')
            return fechanew

        df_facebok_post['cod_postid'] = df_facebok_post['id'].apply(cod_postid)
        df_facebok_post['Fecha_resta'] = pd.to_datetime(df_facebok_post['created_time']) - datetime.timedelta(hours=5)
        df_facebok_post['Fecha'] = pd.to_datetime(df_facebok_post['Fecha_resta']).dt.date
        df_facebok_post['Fecha']= df_facebok_post['Fecha'].apply(quitar_guion)
        df_facebok_post['Hora'] = pd.to_datetime(df_facebok_post['Fecha_resta']).dt.time
        df_facebok_post['Hora'] = df_facebok_post['Hora'].apply(str)
        
        df_facebok_post
        df_facebok_post.isnull().sum()
        df_facebok_post.dtypes
        #Sirve para insertar datos en la tabla FACEBOOK_POST
        currDate = datetime.datetime.now()

        #Insertar en Post
        cursor = conn.cursor()  
        try:
            conn.autocommit = False
            for index,row in df_facebok_post.iterrows(): 
                #history_post_exist =  history_facebook_post[(history_facebook_post.cod_postid == row.cod_postid)]

                #if history_post_exist.empty:
                try:
                    print(row.cod_postid)
                    print('--------')
                    print(row.likes['summary']['total_count'])
                    cursor.execute("UPDATE [IPE.DW].[dbo].[Informe.Facebook_post] SET FbLikes = ?,FBShares = ? WHERE cod_postid = ?", row.likes['summary']['total_count'],row['shares']['count'], row.cod_postid)
   
                        #'summary': {'total_count': 15,
                        #cursor.execute("INSERT INTO [IPE.DW].[dbo].[Informe.Facebook_post] (id,cod_postid,Cliente,Mensaje,Valoracion,Medio,Tipo,Followers,Nombre,Categoria1,Categoria2,Categoria3,URL,Reporte,Fecha,Hora,FBShares,Departamento,Distrito,Org,Sexo,Rango_edad,Influencia,Titulo,Tag1,Tag2,Tag3,Tag4,Tag5,UserID,screen_name,friends,created_at,FbLikes,inreplyto,inreplytouserid,inreplytousername,postid,Lat,Lng,ARCHIVO,Flag_Postexterno) VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)",
                        #   row.cod_postid,row.cod_postid,Cliente,row.message,'','Facebook',row.status_type,0,row['from']['name'],'','','',row.permalink_url,0,row.Fecha,row.Hora,row['shares']['count'],'','','','','','','','','','','','',row['from']['id'],'','',currDate,row['likes']['count'],'','','',row.id,'','','Facebook-Api','1')

                except Exception as e:
                    print('e')
                    print(e)
                #else:
                #    print(row.cod_postid)

        except pyodbc.DatabaseError as err:
            conn.rollback()
        else:
            conn.commit()
        finally:
            conn.autocommit = True        
        cursor.close()